In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Leemos el dataset y configuramos los tipos de dato de cada columna

In [7]:
df = pd.read_csv('./src/AccidentesAvionesLimpio.csv')

df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df[['Ubicación', 'Operador', 'Ruta', 'Tipo de aeronave', 'Resumen']] = df[['Ubicación', 'Operador', 'Ruta', 'Tipo de aeronave', 'Resumen']].astype('string')
df[['Total a bordo', 'Pasajeros a bordo', 'Tripulación a bordo','Total fallecidos', 'Fallecidos pasajeros ', 'Fallecidos tripulación','Fallecidos en tierra']]\
    = df[['Total a bordo', 'Pasajeros a bordo', 'Tripulación a bordo','Total fallecidos', 'Fallecidos pasajeros ', 'Fallecidos tripulación','Fallecidos en tierra']].astype('Int64')

In [20]:
df['Resumen'].fillna('', inplace=True)